In [ ]:
# starting the process of acquiring data from wikipedia 

In [1]:
import mwclient
import time

site = mwclient.Site('en.wikipedia.org')
page = site.pages['Bitcoin']

In [2]:
revs = list(page.revisions())

In [3]:
revs[0]

OrderedDict([('revid', 1154294052),
             ('parentid', 1154267211),
             ('minor', ''),
             ('user', 'Finnusertop'),
             ('timestamp',
              time.struct_time(tm_year=2023, tm_mon=5, tm_mday=11, tm_hour=14, tm_min=25, tm_sec=53, tm_wday=3, tm_yday=131, tm_isdst=-1)),
             ('comment', 'cs1')])

In [4]:
revs = sorted(revs, key=lambda rev: rev["timestamp"]) 

In [5]:
revs[0]

OrderedDict([('revid', 275832581),
             ('parentid', 0),
             ('user', 'Pratyeka'),
             ('timestamp',
              time.struct_time(tm_year=2009, tm_mon=3, tm_mday=8, tm_hour=16, tm_min=41, tm_sec=7, tm_wday=6, tm_yday=67, tm_isdst=-1)),
             ('comment', 'creation (stub)')])

In [6]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

def find_sentiment(text):
    sent = sentiment_pipeline([text[:250]])[0]
    score = sent["score"]
    if sent["label"] == "NEGATIVE":
        score *= -1
    return score

c:\Users\young\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
c:\Users\young\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\young\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HU

In [7]:
edits = {}

for rev in revs:        
    date = time.strftime("%Y-%m-%d", rev["timestamp"])
    if date not in edits:
        edits[date] = dict(sentiments=list(), edit_count=0)
    
    edits[date]["edit_count"] += 1
    
    comment = rev.get("comment", "")
    edits[date]["sentiments"].append(find_sentiment(comment))

In [8]:
from statistics import mean

for key in edits:
    if len(edits[key]["sentiments"]) > 0:
        edits[key]["sentiment"] = mean(edits[key]["sentiments"])
        edits[key]["neg_sentiment"] = len([s for s in edits[key]["sentiments"] if s < 0]) / len(edits[key]["sentiments"])
    else:
        edits[key]["sentiment"] = 0
        edits[key]["neg_sentiment"] = 0
    
    del edits[key]["sentiments"]

In [9]:
import pandas as pd

edits_df = pd.DataFrame.from_dict(edits, orient="index")

In [10]:
edits_df

,edit_count,sentiment,neg_sentiment
2009-03-08,4,-0.550525,0.75
2009-08-05,1,0.748121,0.00
2009-08-06,2,0.995746,0.00
2009-08-14,1,0.930022,0.00
2009-10-13,2,-0.227499,0.50
...,...,...,...
2023-03-30,1,-0.997922,1.00
2023-04-03,1,-0.996604,1.00
2023-04-11,2,0.000964,0.50
2023-04-22,1,-0.998953,1.00


In [11]:
edits_df.index = pd.to_datetime(edits_df.index)

In [12]:
from datetime import datetime

dates = pd.date_range(start="2009-03-08",end=datetime.today())

In [13]:
edits_df = edits_df.reindex(dates, fill_value=0)

In [14]:
edits_df

,edit_count,sentiment,neg_sentiment
2009-03-08,4,-0.550525,0.75
2009-03-09,0,0.000000,0.00
2009-03-10,0,0.000000,0.00
2009-03-11,0,0.000000,0.00
2009-03-12,0,0.000000,0.00
...,...,...,...
2023-05-11,2,-0.005567,0.50
2023-05-12,0,0.000000,0.00
2023-05-13,0,0.000000,0.00
2023-05-14,0,0.000000,0.00


In [ ]:
# now we will perform the rolling edit function for 30 days

In [15]:
rolling_edits = edits_df.rolling(30, min_periods=30).mean()

In [16]:
rolling_edits = rolling_edits.dropna()

In [17]:
rolling_edits

,edit_count,sentiment,neg_sentiment
2009-04-06,0.133333,-0.018351,0.025
2009-04-07,0.000000,0.000000,0.000
2009-04-08,0.000000,0.000000,0.000
2009-04-09,0.000000,0.000000,0.000
2009-04-10,0.000000,0.000000,0.000
...,...,...,...
2023-05-11,0.100000,-0.033484,0.050
2023-05-12,0.100000,-0.033484,0.050
2023-05-13,0.100000,-0.033484,0.050
2023-05-14,0.100000,-0.033484,0.050


In [18]:
rolling_edits.to_csv("wikipedia_edits.csv")

In [ ]:
# now we will get the price data with respect to this wiki data ....